# Landslide4Sense data inspection

In [2]:
 # Imports

import os
import numpy as np

import os
import h5py
import torch
from torch.utils.data import Dataset,DataLoader
import utils
import models
import evalmetrics
import torch.nn as nn


In [7]:
# Visualising the Dataset

# Reading folder of .h5 files images and labels

# Path to the folder containing the .h5 files
path = '/Users/vihaan/Workspace/Local_AI_Earth/Semantic_Segmentation/L4S_Data/TrainData/'

# List of .h5 images
image_directory_path = path + 'img' 

# List of .h5 labels
labels_directory_path = path + 'mask'

# Loading one of the files from image_directory_path h5 files
image_file = h5py.File(image_directory_path + '/image_1.h5', 'r')

# Loading one of the files from labels_directory_path h5 files
labels_file = h5py.File(labels_directory_path + '/mask_1.h5', 'r')

# Printing the keys of the image_file
print(image_file.keys())

# Printing the keys of the labels_file
print(labels_file.keys())

# Loading as torch tensor
image = torch.from_numpy(image_file['img'][()])

print(image.shape)


<KeysViewHDF5 ['img']>
<KeysViewHDF5 ['mask']>
torch.Size([128, 128, 14])


In [ ]:
# Reading a single image

In [3]:
# Splitting the train folder into train, validation and test folders

full_dataset = utils.CustomDataset_L4S_Split_Train_Test_Val(img_dir=path + 'img', mask_dir=path + 'mask')

train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size])

batch_size = 16
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [4]:
torch.cuda.empty_cache()

# Making mpu as device in torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#device = torch.device("cpu")

mps


In [3]:
model = models.EdgeU1_Net(img_ch=14,output_ch=1)

In [4]:
print(model.__class__.__name__)

EdgeU1_Net


In [6]:
# Preparing the model for training

# Defining  loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
#criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 30

######################## Training model on train dataset #####################################

# Train the model
model.to(device)

#Tracking Losses
losst_list = []
lossv_list = []

for epoch in range(num_epochs):
    train_loss = 0.0
    
    #Train the model
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        
        inputs = inputs.float()
        inputs = inputs.to(device)
        #print(inputs.shape)
        labels = labels.to(device)
        #print('labels are')
        #print(labels.shape)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and update
        loss.backward()
        optimizer.step()

        # Print statistics
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)
    losst_list.append(avg_train_loss)
            
    #Validate the training process
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.float()
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader)
    lossv_list.append(avg_val_loss)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])
torch.Size([16, 14, 128, 128])


KeyboardInterrupt: 